In [1]:
import json
import pandas as pd
import pyspark.sql.dataframe
from pca import pca
from pyspark.sql.functions import max, countDistinct, when, rank, lit, pandas_udf, PandasUDFType
from pyspark.sql.window import Window
from typing import List, Dict, Union, Tuple, Optional, Any
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import ClusterCentroids

In [2]:
import warnings
import os
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'
warnings.filterwarnings('ignore')

spark = SparkSession.builder \
    .appName("pandas_udf") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .config("spark.scheduler.mode", "FAIR") \
    .config('spark.driver.memory', '4g') \
    .config('spark.driver.cores', '10') \
    .config('spark.executor.memory', '4g') \
    .config('spark.executor.cores', '10') \
    .config('spark.cores.max', '10') \
    .config('spark.driver.host', '192.168.22.28') \
    .master("spark://192.168.12.47:7077,192.168.12.48:7077") \
    .getOrCreate()

[scatterd] >WARNING> 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [94]:
# from sqlalchemy import create_engine
# engine = create_engine("mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(user="root", password="123456",
#                                                                                          host="192.168.13.17",
#                                                                                          port=9030,
#                                                                                          db="rca"))

In [93]:
# query = "SELECT * FROM rca.DWD_FD_UVA_DATA WHERE START_TIME >= '2021-12-05 14:41:26' AND START_TIME <= '2024-03-05 14:41:26' AND PRODUCT_ID in ('AMKNS301N.0A01','AMKNS301N.0B01','AFPNR901N.0B0J','AFPNR901N.0B0L')"

In [96]:
# operno = ["2V.PQW10","2U.PQA10","1V.PQA10","1U.CDG20","1U.CDG10","3U.PQA10","6V.CDG10","7U.PQA10","7U.PQX10","TM.PQX10","XX.PQW01","XX.PQX02","1U.PQX10","1V.PQX10","1V.PQX20","2U.PQX10","6V.PQA10","3U.PQX10","7U.PQW10","PV.PQX10","TV.PQA10","1C.PQA10","1C.PQX10","7U.CDG10","PV.PQW10","1C.CDG10","1U.PQA10","2V.PQA10","3U.CDG10","3U.CDG20","3U.PQW10","6V.CDG20","6V.EQA10","6V.PQX10","7U.EQA10","7U.EQA20","7U.PQA20","PV.CDG10","PV.EQA10","PV.PQA10","TV.CDG10","TV.EQA20","TV.PQX10","XX.EQW01","2V.PQX10","2V.PQX20","6V.PQW10","7U.ECU10","7U.EQW10","TM.EQA10","1F.FQE10","TV.PQW10","TM.PQW10","TM.PQA10","XX.PQX01","1F.EEK10","2F.CDS10"]
# operno = tuple([string.replace('"', "'") for string in operno])

# prodg1 = ["TESTPRODG1","L11PN03B","L11HH01A","L11HD11A","L11PN06B","L11MW20A","L15EG04A","L11CB14A","L15DV07A","C90WA20A","L11CG02A","L11HH03A","C90WA15A","L15KD03A","C90WA02P","L11PN29B","L15DP03A","L11CR02A","L2800Z3N","L2800Z1N","L2800Z2N","C90WA12A","L11EG07A","L11TG07A","L15RB03A","L11CS04A","C90WA30A","L11DM10A","L11CD02A","L11PP03A","L11TG05A"]
# prodg1 = tuple([string.replace('"', "'") for string in prodg1])

# productid = ["AFPNM301N.0A01","AFPNR901N.0B01","AFKNNW01N.0C01","AMKNS301N.0A01","AFGN2T01N.0G01","AEMNE801N.0B01","AEMNV901N.0A01","AMKNWX01N.0B01","AMKNZC01N.0A01","AMKNSE01N.0B01","AMKNS301N.0B01","AFGN4201N.0B01","AMKNTJ01N.0A01","AMKNZD01N.0A01","AFKN8401N.0D01"]
# productid = tuple([string.replace('"', "'") for string in productid])

# eqp = ["DSA02","EKT72","TEST01"]
# eqp = tuple([string.replace('"', "'") for string in eqp])

# tool = ["TEST01_A","EKT72_PM1","DSA02_B"]
# tool = tuple([string.replace('"', "'") for string in tool])

# recipe = ["","NEW-DRM/P1/110NM/PFKN0G0D1F1A","S180DE5C680-2X","NEW-DRM/P1/110NM/PFKN9F0D1F1A","NEW-DRM/P1/150NM/PFGN420D1F1A","NEW-DRM/P1/110NM/PFKN560D1F1A","NEW-DRM/P1/110NM/PFKN690D1F1A","NEW-DRM/P1/110NM/PFKN6N0D1F1A","NEW-DRM/P1/110NM/PFKN050D1F1A","S8900DX0580","NEW-DRM/P1/150NM/PFGN2T0D1F1A","NEW-DRM/P1/110NM/PFKN750D1F1A"]
# recipe = tuple([string.replace('"', "'") for string in recipe])

# query = f"SELECT * FROM rca.DWD_FD_UVA_DATA WHERE START_TIME >= '2021-12-05 16:14:52'\
# AND START_TIME <= '2024-03-05 16:14:52'\
# AND OPER_NO in {operno}\
# AND PRODG1 in {prodg1}\
# AND PRODUCT_ID in {productid}\
# AND EQP_NAME in {eqp}\
# AND TOOL_NAME in {tool}\
# AND RECIPE_NAME in {recipe}"

# data_pandas = pd.read_sql(sql=query, con=engine)

In [97]:
# query_wafer = "SELECT * FROM rca.conf_wafer WHERE upload_id = '767d668b11664c47a04903b246994437'"
# data_wafer = pd.read_sql(sql=query_wafer, con=engine)
# data_wafer

In [78]:
# data_wafer['label'] = data_wafer['label'].map({'BAD':1, 'GOOD':0})
# data_wafer.rename({'name':'WAFER_ID'}, axis='columns', inplace=True)

In [98]:
# data_wafer[data_wafer['label']==1]

In [99]:
# 数据和label连接
# data_pandas_label = pd.merge(data_pandas, data_wafer[['WAFER_ID', 'label']])
# data_pandas_label[data_pandas_label['label']==1]['WAFER_ID'].unique()

In [3]:
pandas_df = pd.read_csv("C:/Users/yang.wenjun/Desktop/dd1.csv")

In [4]:
pandas_df

,WAFER_ID,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,LOT_ID,RECIPE_NAME,OPER_NO,parametric_name,CASE_INFO,STATUS,STATISTIC_RESULT,label
0,NBX293-06,14847,125676,WKL5A,AFPNR901N.0B0L,L2800Z2N,WKL5A_L1,NBX293.200,KFPM10A.PrD,1V.WWK20,CHUCK_ROTATION_SPEED#WINDOW_1#MEAN,2023/12/25,NORMAL,1483,0
1,NBX293-06,14847,125676,WKL5A,AFPNR901N.0B0L,L2800Z2N,WKL5A_L1,NBX293.200,KFPM10A.PrD,1V.WWK20,DIW_FLOW#WINDOW_1#MEAN,2023/12/25,NORMAL,1511,0
2,NBX293-06,14847,125676,WKL5A,AFPNR901N.0B0L,L2800Z2N,WKL5A_L1,NBX293.200,KFPM10A.PrD,1V.WWK20,FPM_CIRCULATION_FLOW#WINDOW_1#MEAN,2023/12/25,NORMAL,21,0
3,NBX293-06,14847,125676,WKL5A,AFPNR901N.0B0L,L2800Z2N,WKL5A_L1,NBX293.200,KFPM10A.PrD,1V.WWK20,PSL_PM1_CHUCKPINWAFERCOUNT#WINDOW_1#MEAN,2023/12/25,NORMAL,5906,0
4,NBX293-06,14847,125676,WKL5A,AFPNR901N.0B0L,L2800Z2N,WKL5A_L1,NBX293.200,KFPM10A.PrD,1V.WWK20,FPM_TEMP_BEFORE_HEATER#NEW#MEAN,2023/12/25,NORMAL,23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25271,NBX293-06,7046,170461,EML01,AFPNR901N.0B0L,L2800Z2N,EML01_PM3,NBX293.200,RJPL28M2U1T01R02,2U.EEM10,BASI_RF_VOLTAGE#AUTO_CURRENT_STEP_9#MEAN,2023/12/8,NORMAL,2,0
25272,NBX293-06,7046,170461,EML01,AFPNR901N.0B0L,L2800Z2N,EML01_PM3,NBX293.200,RJPL28M2U1T01R02,2U.EEM10,BIAS_MATCH_SHUNT_POSITION#AUTO_CURRENT_STEP_9#...,2023/12/8,NORMAL,686,0
25273,NBX293-06,7046,170461,EML01,AFPNR901N.0B0L,L2800Z2N,EML01_PM3,NBX293.200,RJPL28M2U1T01R02,2U.EEM10,ESC_TEMPERATURE_INNTER#AUTO_CURRENT_STEP_9#MEAN,2023/12/8,NORMAL,48,0
25274,NBX293-06,7046,170462,EML01,AFPNR901N.0B0L,L2800Z2N,EML01_PM3,NBX293.200,Tier2 WAC_Fluorine,2U.EEM10,GAS8_CL2_500#WINDOW_1#AREA,2023/12/8,NORMAL,3334,0


#### uva算法代码

In [5]:
df1_ = ps.from_pandas(pandas_df).to_spark()
print(df1_.count())

25276


In [6]:
class PreprocessForUvaData:
    @staticmethod
    def integrate_columns(df: pyspark.sql.dataframe,
                          merge_operno_list: List[Dict[str, List[str]]],
                          merge_prodg1_list: List[Dict[str, List[str]]],
                          merge_product_list: List[Dict[str, List[str]]],
                          merge_eqp_list: List[Dict[str, List[str]]],
                          merge_chamber_list: List[Dict[str, List[str]]], ) -> pyspark.sql.dataframe:
        """
        Integrate columns in the DataFrame based on the provided list.

        :param df: The input DataFrame.
        :param merge_operno_list: A list of dictionaries where each dictionary contains values to be merged.
               Example: [{'2F.CDS10_XX.TDS01': ['2F.CDS10', 'XX.TDS01']},
                         {'2F.CDS20_XX.CDS20': ['2F.CDS20', 'XX.CDS20']}]
        :param merge_prodg1_list: A list of dictionaries for merging 'PRODG1' column in a similar fashion.
        :param merge_product_list: A list of dictionaries for merging 'PRODUCT_ID' column in a similar fashion.
        :param merge_eqp_list: A list of dictionaries for merging 'EQP_NAME' column in a similar fashion.
        :param merge_chamber_list: A list of dictionaries for merging 'TOOL_NAME' column in a similar fashion.

        :return: DataFrame with 'OPER_NO' and other specified columns integrated according to the merge rules.
        """
        if merge_operno_list is not None and len(merge_operno_list) > 0:
            # Extract values from each dictionary in merge_operno_list and create a list
            values_to_replace = [list(rule.values())[0] for rule in merge_operno_list]
            # Concatenate values from each dictionary
            merged_values = ["_".join(list(rule.values())[0]) for rule in merge_operno_list]

            # Replace values in 'OPER_NO' column based on the rules defined in merge_operno_list
            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn("OPER_NO",
                                   when(col("OPER_NO").isin(values), replacement_value).otherwise(col("OPER_NO")))

        if merge_prodg1_list is not None and len(merge_prodg1_list) > 0:
            values_to_replace = [list(rule.values())[0] for rule in merge_prodg1_list]
            merged_values = ["_".join(list(rule.values())[0]) for rule in merge_prodg1_list]

            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn("PRODG1",
                                   when(col("PRODG1").isin(values), replacement_value).otherwise(col("PRODG1")))

        if merge_product_list is not None and len(merge_product_list) > 0:
            values_to_replace = [list(rule.values())[0] for rule in merge_product_list]
            merged_values = ["_".join(list(rule.values())[0]) for rule in merge_product_list]

            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn("PRODUCT_ID",
                                   when(col("PRODUCT_ID").isin(values), replacement_value).otherwise(col("PRODUCT_ID")))

        if merge_eqp_list is not None and len(merge_eqp_list) > 0:
            values_to_replace = [list(rule.values())[0] for rule in merge_eqp_list]
            merged_values = ["_".join(list(rule.values())[0]) for rule in merge_eqp_list]

            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn("EQP_NAME",
                                   when(col("EQP_NAME").isin(values), replacement_value).otherwise(col("EQP_NAME")))

        if merge_chamber_list is not None and len(merge_chamber_list) > 0:
            values_to_replace = [list(rule.values())[0] for rule in merge_chamber_list]
            merged_values = ["_".join(list(rule.values())[0]) for rule in merge_chamber_list]

            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn("TOOL_NAME",
                                   when(col("TOOL_NAME").isin(values), replacement_value).otherwise(col("TOOL_NAME")))
        return df

    @staticmethod
    def pre_process(df: pyspark.sql.dataframe) -> pyspark.sql.dataframe:
        """
        Preprocess the data extracted from the database for a specific CASE.
        :param df: Data for a specific CASE retrieved from the database.
        :return: Preprocessed data with relevant columns and filters applied.
        """
        # Select only the columns that will be used
        df = df.select('WAFER_ID', 'TOOL_ID', 'RUN_ID', 'EQP_NAME', 'PRODUCT_ID', 'PRODG1', 'TOOL_NAME',
                       'OPER_NO', 'parametric_name', 'STATISTIC_RESULT', 'label')
        # Remove rows with missing values in 'STATISTIC_RESULT' column
        df = df.filter(col('STATISTIC_RESULT').isNotNull())
        # Drop duplicates based on all columns
        df1 = df.dropDuplicates()
        # Select the rows with the latest 'RUN_ID' for each combination of 'WAFER_ID', 'OPER_NO', 'TOOL_ID'
        df2 = df1.groupBy('WAFER_ID', 'OPER_NO', 'TOOL_ID').agg(max('RUN_ID').alias('RUN_ID'))
        df_run = df1.join(df2.dropDuplicates(subset=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID']),
                          on=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID'], how='inner')
        return df_run

    @staticmethod
    def commonality_analysis(df_run: pyspark.sql.dataframe, grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Perform commonality analysis on preprocessed data.
        :param df_run: Preprocessed data after data preprocessing.
        :param grpby_list: List of columns ['PRODG1', 'EQP_NAME', 'OPER_NO', 'PRODUCT_ID', 'TOOL_NAME'] for grouping.
                Example: grpby_list = ['PRODG1', 'TOOL_NAME', 'OPER_NO'], grpby_list = ['PRODUCT_ID', 'OPER_NO']
        :return: Results of commonality analysis, showing the top ten combinations with the highest number of bad wafers.
        """
        grps = (df_run.groupBy(grpby_list)
                .agg(countDistinct('WAFER_ID').alias('wafer_count'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
                .orderBy('bad_num', ascending=False))

        # Handle the case of a single OPER_NO or single TOOL_NAME
        if grps.count() == 1:
            return grps
        else:
            # Filter out groups with no bad wafers
            grps = grps.filter(grps['bad_num'] > 0)
            # Rank the groups based on the number of bad wafers
            window_sep = Window().orderBy(col("bad_num").desc())
            ranked_df = grps.withColumn("rank", rank().over(window_sep))
            # Select the top ten groups and remove the 'rank' column
            grpss = ranked_df.filter(col("rank") <= 10).drop("rank")
            return grpss

    @staticmethod
    def get_data_list(common_res: pyspark.sql.dataframe,
                      grpby_list: List[str],
                      big_or_small: str = 'big') -> List[Dict[str, str]]:
        """
        Get a list of dictionaries for corresponding groups based on commonality analysis.

        :param common_res: Result of commonality analysis.
        :param grpby_list:  List of columns ['PRODG1', 'EQP_NAME', 'OPER_NO', 'PRODUCT_ID', 'TOOL_NAME'] for grouping.
        :param big_or_small: 'big' or 'small'.
        :return: List of dictionaries for corresponding groups.
                Example: [{'OPER_NO': '1F.EEK10', 'PRODUCT_ID': 'AFKN2J01N.0U01'},
                          {'OPER_NO': '1F.EEK10', 'PRODUCT_ID': 'AFKN4X01N.0B01'},
                          {'OPER_NO': '1F.EEK10', 'PRODUCT_ID': 'AFGN1501N.0C02'}]
        """
        assert big_or_small in ['big', 'small'], "Choose only 'big' or 'small'. Please check the spelling."

        # Filter groups based on big or small sample conditions
        if big_or_small == 'big':
            good_bad_grps = common_res.filter("good_num >= 3 AND bad_num >= 3")
        else:
            good_bad_grps = common_res.filter("bad_num >= 1 AND wafer_count >= 2")

        # Order the results and limit to the top 10 groups
        good_bad_grps = good_bad_grps.orderBy(col("bad_num").desc(), col("wafer_count").desc(),
                                              col("good_num").desc()).limit(10)

        # Collect the data and convert it into a list of dictionaries
        data_list = good_bad_grps[grpby_list].collect()
        data_dict_list = [row.asDict() for row in data_list]
        return data_dict_list

    @staticmethod
    def get_train_data(df_run: pyspark.sql.dataframe, data_dict_list: List[Dict[str, str]]) -> pyspark.sql.dataframe:
        """
        Get the actual combination data for modeling from the original data.

        :param df_run: Preprocessed data after data preprocessing.
        :param data_dict_list: List of dictionaries with filtering conditions.
               Example: [{'OPER_NO': '1F.EEK10', 'PRODUCT_ID': 'AFKN2J01N.0U01'},
                          {'OPER_NO': '1F.EEK10', 'PRODUCT_ID': 'AFKN4X01N.0B01'},
                          {'OPER_NO': '1F.EEK10', 'PRODUCT_ID': 'AFGN1501N.0C02'}]
        :return: Filtered data for modeling.
        """
        # Get the filtering conditions for the first data dictionary
        first_data_dict = data_dict_list[0]
        conditions = " AND ".join(["{} == '{}'".format(col_, first_data_dict[col_]) for col_ in first_data_dict])
        # Filter the data for the first condition
        df_s = df_run.filter(conditions)

        # Loop through the remaining data dictionaries and filter the data accordingly
        for i in range(1, len(data_dict_list)):
            data_dict = data_dict_list[i]
            conditions = " AND ".join(["{} == '{}'".format(col_, data_dict[col_]) for col_ in data_dict])
            df_m = df_run.filter(conditions)
            df_s = df_s.union(df_m)
        return df_s

    @staticmethod
    def get_all_bad_wafer_num(df: pyspark.sql.dataframe) -> int:
        """
        Get the number of distinct bad WAFER in the DataFrame.
        """
        return df.filter("label == 1").select('WAFER_ID').distinct().count()


class FitModelForUvaData:
    @staticmethod
    def get_pivot_table(df: pd.DataFrame, grpby_list: List[str]) -> pd.DataFrame:
        """
        Pivot the DataFrame based on specified grouping columns.

        Parameters:
        - df: Data for modeling.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Result of pivoting the table.
        """
        index_cols = ['WAFER_ID', 'label']
        columns_cols = grpby_list + ['parametric_name']
        df_pivot = df.dropna(axis=0).pivot_table(index=index_cols,
                                                 columns=columns_cols,
                                                 values=['STATISTIC_RESULT'])
        df_pivot.columns = df_pivot.columns.map('#'.join)
        df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
        # Remove completely identical columns
        for column in df_pivot.columns.difference(index_cols):
            if df_pivot[column].nunique() == 1:
                df_pivot = df_pivot.drop(column, axis=1)
        return df_pivot

    @staticmethod
    def fit_rf_big_sample(df: pyspark.sql.dataframe, grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Fit a RandomForest model on the train data. It is for large sample method(good_wafer_num >= 3 AND bad_wafer_num >= 3)

        Parameters:
        - df: Data for modeling.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Combined dataframe of roc_auc_score result and feature importance after RandomForest modeling.
        """
        # Dynamically build schema according to the grpby_list
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("bad_wafer", IntegerType(), True),
                              StructField("roc_auc_score", FloatType(), True),
                              StructField("features", StringType(), True),
                              StructField("importance", FloatType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run: pd.DataFrame) -> pd.DataFrame:
            # Pivot the table
            df_pivot = FitModelForUvaData.get_pivot_table(df=df_run, grpby_list=grpby_list)

            # Define independent and dependent variables
            x_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]
            y_train = df_pivot[['label']]
            if min(x_train.shape) <= 0:
                return pd.DataFrame()

            z_ratio = y_train.value_counts(normalize=True)
            good_ratio = z_ratio[0]
            bad_ratio = z_ratio[1]
            if abs(good_ratio - bad_ratio) > 0.7:
                undersampler = ClusterCentroids(random_state=101)
                x_train, y_train = undersampler.fit_resample(x_train, y_train)

            # Grid search
            pipe = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
                ('scaler', StandardScaler()),
                ('model', RandomForestClassifier(random_state=2024))])
            param_grid = {'model__n_estimators': [*range(50, 100, 10)],
                          'model__max_depth': [*range(10, 50, 10)]}
            grid = GridSearchCV(estimator=pipe, scoring='roc_auc', param_grid=param_grid, cv=3, n_jobs=-1)
            grid.fit(x_train.values, y_train.values.ravel())
            roc_auc_score_ = grid.best_score_

            # Feature importance and result summary
            small_importance_res = pd.DataFrame({'features': x_train.columns,
                                                 'importance': grid.best_estimator_.steps[2][1].feature_importances_})

            sample_res_dict = {'bad_wafer': sum(df_pivot['label']),
                               'roc_auc_score': roc_auc_score_}
            sample_res_dict.update({col_: df_run[col_].unique() for col_ in grpby_list})
            small_sample_res = pd.DataFrame(sample_res_dict)
            return pd.concat([small_importance_res, small_sample_res])

        return df.groupby(grpby_list).apply(get_model_result)

    @staticmethod
    def fit_pca_small_sample(df: pyspark.sql.dataframe, grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Fit a PCA model on the train data. It is for small sample method (bad_wafer_num >= 1 AND wafer_count >= 2).

        Parameters:
        - df: Data for modeling.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Combined dataframe of every feature and its importance in each combination of grpby_list after PCA modeling.
        """
        # Dynamically build schema according to the grpby_list
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("features", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("bad_wafer", IntegerType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run: pd.DataFrame) -> pd.DataFrame:
            """
            Perform PCA modeling on the small sample data.

            Parameters:
            - df_run: Subset of data for modeling (pandas.DataFrame).

            Returns:
            - pd.DataFrame: Combined dataframe of every feature and its importance in each combination of grpby_list after PCA modeling.
            """
            df_pivot = FitModelForUvaData.get_pivot_table(df=df_run, grpby_list=grpby_list)
            # Since it is a small sample, make a copy to generate more data for the PCA model
            df_pivot_copy = df_pivot.copy()
            df_pivot_all = pd.concat([df_pivot, df_pivot_copy], axis=0)

            # Define independent variables
            x_train = df_pivot_all[df_pivot_all.columns.difference(['WAFER_ID', 'label']).tolist()]
            if min(x_train.shape) <= 0:
                return pd.DataFrame()

            n_components = min(min(x_train.shape) - 2, 20)
            model = pca(n_components=n_components, verbose=None)
            results = model.fit_transform(x_train)
            res_top = results['topfeat']
            res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
            res_top_select['importance'] = abs(res_top_select['loading'])
            res_top_select = res_top_select.rename(columns={'feature': 'features'}).drop("loading",
                                                                                         axis=1).drop_duplicates()

            # Add some field information
            res_top_select['bad_wafer'] = sum(df_pivot['label'])
            for col_ in grpby_list:
                res_top_select[col_] = df_run[col_].values[0]
            return res_top_select

        return df.groupby(grpby_list).apply(get_model_result)


class GetFinalResultsForUvaData:
    @staticmethod
    def split_score_big_sample(df: pyspark.sql.dataframe, grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Split the ROC AUC scores based on the specified grouping columns.

        Parameters:
        - df: Results after RandomForest modeling.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: ROC AUC scores result with each element in grpby_list as columns.
        """
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("bad_wafer", IntegerType(), True),
                              StructField("roc_auc_score", FloatType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(model_results: pd.DataFrame) -> pd.DataFrame:
            select_expr = grpby_list + ['bad_wafer', 'roc_auc_score']
            sample_res = model_results[select_expr].dropna(axis=0)
            sample_res = sample_res[sample_res['roc_auc_score'] > 0.6]
            return sample_res

        return df.groupby(grpby_list).apply(get_result)

    @staticmethod
    def split_features(df: pd.DataFrame, index: int) -> str:
        """
        Split the 'features' column based on the specified index.

        Parameters:
        - df: RandomForest modeling results with 'features' column.
        - index: Order value.

        Returns:
        - str: Field attribute value.
        """
        return df['features'].apply(lambda x: x.split('#')[index])

    @staticmethod
    def get_split_feature_importance_table(df: pd.DataFrame, grpby_list: List[str]) -> pd.DataFrame:
        """
        Get the table after splitting the 'features' column based on the specified grouping columns.

        Parameters:
        - df: RandomForest modeling results with 'features' column.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Table after splitting features.
        """
        n_feats = len(grpby_list)
        for i in range(n_feats):
            df[grpby_list[i]] = GetFinalResultsForUvaData.split_features(df, i + 1)

        df['parametric_name'] = GetFinalResultsForUvaData.split_features(df, n_feats + 1)
        df['step'] = GetFinalResultsForUvaData.split_features(df, n_feats + 2)
        df['stats'] = GetFinalResultsForUvaData.split_features(df, n_feats + 3)
        df = df.drop(['features'], axis=1).reset_index(drop=True)
        return df

    @staticmethod
    def add_feature_stats(df: pd.DataFrame, grpby_list: List[str]) -> pd.DataFrame:
        """
        Add a column with all statistical features of parameters.

        Parameters:
        - df: Feature importance table after processing.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: New column containing all statistical features: 'feature_stats'.
        """
        grpby_list_extend = grpby_list + ['parametric_name', 'step']
        feature_stats = df.groupby(grpby_list_extend)['stats'].unique().reset_index()
        feature_stats['stats'] = [feature_stats['stats'].iloc[i].tolist() for i in range(len(feature_stats))]
        feature_stats['stats'] = feature_stats['stats'].apply(lambda x: "#".join(x))
        feature_stats = feature_stats.assign(
            parametric_name=lambda x: x['parametric_name'] + str('#') + x['step']).drop(
            'step', axis=1)
        return feature_stats

    @staticmethod
    def split_calculate_features_big_sample(df: pyspark.sql.dataframe, grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Split and calculate features based on the specified grouping columns.

        Parameters:
        - df: Results after RandomForest modeling.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Features importance results.
        """
        # Dynamically build schema
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("parametric_name", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("stats", StringType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(model_results: pd.DataFrame) -> pd.DataFrame:
            # Extract 'features' and 'importance' from the RandomForest model results
            feature_importance_table = model_results[['features', 'importance']].dropna(axis=0)

            # Split features
            feature_importance_res_split = GetFinalResultsForUvaData.get_split_feature_importance_table(
                df=feature_importance_table,
                grpby_list=grpby_list)

            # Remove combinations with importance equal to 0
            feature_importance_res_split_drop = feature_importance_res_split.query("importance > 0").reset_index(
                drop=True)

            # Take the top 60% or 100% of each combination result
            feature_importance_res_split_nlargest = (feature_importance_res_split_drop.groupby(by=grpby_list)
                                                     .apply(
                lambda x: x.nlargest(int(x.shape[0] * 0.6), 'importance') if x.shape[0] > 1 else x.nlargest(
                    int(x.shape[0] * 1), 'importance'))
                                                     .reset_index(drop=True))

            # Add a column with all statistical features: 'feature_stats'
            feature_stats = GetFinalResultsForUvaData.add_feature_stats(df=feature_importance_res_split_drop,
                                                                        grpby_list=grpby_list)

            # Sum the importance for the same combination and parameter: 'feature_importance_groupby'
            feature_importance_groupby = (
                feature_importance_res_split_nlargest.groupby(grpby_list + ['parametric_name', 'step'])['importance']
                .sum().reset_index())
            feature_importance_groupby = (
                feature_importance_groupby.assign(parametric_name=lambda x: x['parametric_name'] + str('#') + x['step'])
                .drop('step', axis=1))

            # Connect 'feature_stats' and 'feature_importance_groupby'
            grpby_stats = pd.merge(feature_stats, feature_importance_groupby,
                                   on=grpby_list + ['parametric_name']).dropna().reset_index(drop=True)
            return grpby_stats

        return df.groupby(grpby_list).apply(get_result)

    @staticmethod
    def get_final_results_big_sample(s_res: pyspark.sql.dataframe, f_res: pyspark.sql.dataframe, grpby_list: List[str],
                                     bad_wafer_num: int) -> pyspark.sql.dataframe:
        """
        Get the final modeling results.

        Parameters:
        - s_res: ROC AUC scores result.
        - f_res: Features importance result.
        - grpby_list: List of grouping columns.
        - bad_wafer_num: Number of bad wafers in the data.

        Returns:
        - DataFrame: Final modeling result.
        """
        roc_auc_score_all = s_res.agg({"roc_auc_score": "sum"}).collect()[0][0]
        s_res = s_res.withColumn("roc_auc_score_ratio", col("roc_auc_score") / roc_auc_score_all)
        s_res = s_res.withColumn("bad_ratio", col("bad_wafer") / bad_wafer_num)

        df_merge = s_res.join(f_res, on=grpby_list, how='left')
        df_merge = df_merge.withColumn('weight_original',
                                       col('roc_auc_score_ratio') * col('bad_ratio') * col('importance'))

        # Normalize again
        weight_all = df_merge.agg({"weight_original": "sum"}).collect()[0][0]
        df_merge = df_merge.withColumn("weight", col("weight_original") / weight_all)
        df_merge = df_merge.select(grpby_list + ['parametric_name', 'weight', 'stats']).orderBy('weight',
                                                                                                ascending=False)
        return df_merge

    @staticmethod
    def split_calculate_features_small_sample(df: pyspark.sql.dataframe,
                                              grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Calculate features and importance after PCA modeling on a small sample.

        Parameters:
        - df: PCA modeling results (pyspark.sql.dataframe).
        - grpby_list: List of grouping columns (List[str]).

        Returns:
        - DataFrame: Dataframe containing features, importance and other information after PCA modeling.
        """
        # Dynamically build schema
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("parametric_name", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("bad_wafer", IntegerType(), True),
                              StructField("stats", StringType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(model_results: pd.DataFrame) -> pd.DataFrame:
            feature_importance_table = model_results[['features', 'importance', 'bad_wafer']].dropna(axis=0)
            # Split features
            feature_importance_res_split = GetFinalResultsForUvaData.get_split_feature_importance_table(
                df=feature_importance_table,
                grpby_list=grpby_list)

            # Add a column with all statistical features containing parameters: feature_stats
            feature_stats = GetFinalResultsForUvaData.add_feature_stats(df=feature_importance_res_split,
                                                                        grpby_list=grpby_list)

            # Sum the same parameter in the same combination: feature_importance_groupby
            feature_importance_groupby = (
                feature_importance_res_split.groupby(grpby_list + ['bad_wafer', 'parametric_name', 'step'])[
                    'importance'].sum().reset_index())
            feature_importance_groupby = feature_importance_groupby.assign(
                parametric_name=lambda x: x['parametric_name'] + str('#') + x['step']).drop('step', axis=1)

            # Connect feature_stats and feature_importance_groupby
            grpby_stats = pd.merge(feature_stats, feature_importance_groupby,
                                   on=grpby_list + ['parametric_name']).dropna().reset_index(drop=True)
            return grpby_stats

        return df.groupby(grpby_list).apply(get_result)

    @staticmethod
    def get_final_results_small_sample(f_res: pyspark.sql.dataframe,
                                       bad_wafer_num: int,
                                       grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Get the final modeling results for a small sample.

        Parameters:
        - f_res: Features and importance results (pyspark.sql.dataframe).
        - bad_wafer_num: Total number of bad wafers in the data (int).
        - grpby_list: List of grouping columns (List[str]).

        Returns:
        - DataFrame: Final modeling results with weights and statistics.
        """
        f_res = f_res.withColumn("bad_ratio", col("bad_wafer") / bad_wafer_num)
        df_merge = f_res.withColumn('weight_original', col('importance') * col('bad_ratio'))

        # Normalize weights again
        weight_all = df_merge.agg({"weight_original": "sum"}).collect()[0][0]
        df_merge = df_merge.withColumn("weight", col("weight_original") / weight_all)

        # Select columns and order by weight in descending order
        df_merge = df_merge.select(grpby_list + ['parametric_name', 'weight', 'stats']).orderBy('weight',
                                                                                                ascending=False)
        return df_merge

    @staticmethod
    def add_certain_column(df: pyspark.sql.dataframe, by: str, request_id: str,
                           grpby_list: List[str]) -> pyspark.sql.dataframe:
        """
        Add specific columns to the final modeling results.

        Parameters:
        - df: Final modeling result.
        - by: Grouping column, manually add a column 'add'.
        - request_id: Request ID passed in.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Final modeling result with specific columns added.
        """
        # Dynamically build schema_all
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("stats", StringType(), True),
                              StructField("parametric_name", StringType(), True),
                              StructField("weight", FloatType(), True),
                              StructField("request_id", StringType(), True),
                              StructField("weight_percent", FloatType(), True),
                              StructField("index_no", IntegerType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(final_res: pd.DataFrame) -> pd.DataFrame:
            final_res['weight'] = final_res['weight'].astype(float)
            final_res = final_res.query("weight > 0")
            final_res['request_id'] = request_id
            final_res['weight_percent'] = final_res['weight'] * 100
            final_res = final_res.sort_values('weight', ascending=False)
            final_res['index_no'] = [i + 1 for i in range(len(final_res))]
            final_res = final_res.drop('add', axis=1)
            return final_res

        return df.groupby(by).apply(get_result)


class ExertUvaAlgorithm:
    @staticmethod
    def fit_big_data_model(df_run: pyspark.sql.dataframe,
                           data_dict_list_bs: List[Dict[str, str]],
                           grpby_list: List[str],
                           request_id: str) -> Union[str, pyspark.sql.dataframe.DataFrame]:
        """
        Perform random forest model on a large sample of data(good_wafer_num >= 3 AND bad_wafer_num >= 3).

        Parameters:
        - df_run: Preprocessed data after data preprocessing.
        - data_dict_list_bs: List of dictionaries with filtering conditions for big sample.
        - grpby_list: List of grouping columns.
        - request_id: Unique identifier for the request.

        Returns:
        - DataFrame: Dataframe containing the error message or the final modeling results.
        """
        # 1. Get data for modeling with a large sample
        df_run_bs = PreprocessForUvaData.get_train_data(df_run=df_run, data_dict_list=data_dict_list_bs)
        if df_run_bs.count() == 0:
            msg = 'No data of this type in the database!'
            return msg

        # 2. Get the total number of bad wafers
        bad_wafer_num_big_sample = PreprocessForUvaData.get_all_bad_wafer_num(df=df_run_bs)
        if bad_wafer_num_big_sample < 3:
            msg = 'The actual number of BAD_WAFER in the database is less than 3, please provide more BAD_WAFER!'
            return msg

        # 3. Model the selected big sample data
        res = FitModelForUvaData.fit_rf_big_sample(df=df_run_bs, grpby_list=grpby_list)
        if res.count() == 0:
            msg = 'Temporary algorithm exception!'
            return msg

        # 4. Integrate the modeling results
        s_res = GetFinalResultsForUvaData.split_score_big_sample(df=res, grpby_list=grpby_list)
        if s_res.count() == 0:
            msg = 'The algorithm has a low running score, no output for now, it is recommended to increase the number of BAD_WAFER'
            return msg

        f_res = GetFinalResultsForUvaData.split_calculate_features_big_sample(df=res, grpby_list=grpby_list)
        if f_res.count() == 0:
            msg = 'Temporary exception in calculating the algorithm result'
            return msg

        model_res_bs = GetFinalResultsForUvaData.get_final_results_big_sample(s_res=s_res, f_res=f_res,
                                                                              grpby_list=grpby_list,
                                                                              bad_wafer_num=bad_wafer_num_big_sample)
        if model_res_bs.count() == 0:
            msg = 'Temporary exception in splicing algorithm results'
            return msg

        # 7. Add specific columns
        final_res_bs = model_res_bs.withColumn('add', lit(0))
        final_res_add_columns = GetFinalResultsForUvaData.add_certain_column(df=final_res_bs, by='add',
                                                                             request_id=request_id,
                                                                             grpby_list=grpby_list)
        if final_res_add_columns.count() == 0:
            msg = 'Temporary exception in adding columns to algorithm results'
            return msg
        else:
            return final_res_add_columns

    @staticmethod
    def fit_small_data_model(df_run: pyspark.sql.dataframe,
                             common_res: pyspark.sql.dataframe,
                             grpby_list: List[str],
                             request_id: str) -> Union[str, pyspark.sql.dataframe.DataFrame]:
        """
        Perform PCA model on a small sample of data (bad_wafer_num >= 1 AND wafer_count >= 2).

        Parameters:
        - df_run: Preprocessed data after data preprocessing.
        - common_res: Common analysis results dataframe.
        - grpby_list: List of grouping columns.
        - request_id: Unique identifier for the request.

        Returns:
        - Union[str, DataFrame]: Error message or the final modeling results dataframe.
        """
        # 1. Get data for modeling with a large sample
        data_dict_list_ss = PreprocessForUvaData.get_data_list(common_res=common_res, grpby_list=grpby_list,
                                                               big_or_small='small')
        if len(data_dict_list_ss) == 0:
            msg = 'The actual number of WAFER in the database under this query condition is 1, unable to analyze'
            return msg

        # 2. Get the data
        df_run_ss = PreprocessForUvaData.get_train_data(df_run=df_run, data_dict_list=data_dict_list_ss)
        if df_run_ss.count() == 0:
            msg = 'No data of this type in the database!'
            return msg

        # 3. Get the total number of bad wafers
        bad_wafer_num_small_sample = PreprocessForUvaData.get_all_bad_wafer_num(df_run_ss)
        if bad_wafer_num_small_sample <= 1:
            msg = 'The actual number of BAD_WAFER in the database under this query condition is 1, please provide more BAD_WAFER!'
            return msg

        # 4. Model the selected big sample data
        res = FitModelForUvaData.fit_pca_small_sample(df=df_run_ss, grpby_list=grpby_list)
        if res.count() == 0:
            msg = 'Temporary algorithm exception!'
            return msg

        f_res = GetFinalResultsForUvaData.split_calculate_features_small_sample(df=res, grpby_list=grpby_list)
        if f_res.count() == 0:
            msg = 'Temporary exception in calculating the algorithm result'
            return msg

        model_res_ss = GetFinalResultsForUvaData.get_final_results_small_sample(f_res=f_res,
                                                                                bad_wafer_num=bad_wafer_num_small_sample,
                                                                                grpby_list=grpby_list)
        if model_res_ss.count() == 0:
            msg = 'Temporary exception in splicing algorithm results'
            return msg

        final_res_ss = model_res_ss.withColumn('add', lit(0))
        final_res_add_columns = GetFinalResultsForUvaData.add_certain_column(df=final_res_ss, by='add',
                                                                             request_id=request_id,
                                                                             grpby_list=grpby_list)
        if final_res_add_columns.count() == 0:
            msg = 'Temporary exception in adding columns to algorithm results'
            return msg
        else:
            return final_res_add_columns

    @staticmethod
    def validate_grpby_list(grpby_list: List[str]) -> bool:
        valid_fields = {'PRODUCT_ID', 'OPER_NO', 'EQP_NAME', 'PRODG1', 'TOOL_NAME'}
        for field in grpby_list:
            if field not in valid_fields:
                return False
        return True

    @staticmethod
    def get_some_info(df: pd.DataFrame) -> tuple[
        Any, Any, Any, Optional[list[Any]], Optional[list[Any]], Optional[list[Any]], Optional[list[Any]], Optional[
            list[Any]]]:
        """
        Extracts information from a DataFrame containing request information.

        Parameters:
        - df: DataFrame containing request information.

        Returns:
        - Tuple containing parsed information.
        """
        request_id = df["requestId"].values[0]
        request_params = df["requestParam"].values[0]
        request_params = request_params.replace('\'', '\"')

        parse_dict = json.loads(request_params)
        grpby_list = parse_dict[0]['grpby_list']
        if not ExertUvaAlgorithm.validate_grpby_list(grpby_list):
            raise ValueError(
                "Invalid grpby_list. Must include at least one of 'PRODUCT_ID', 'OPER_NO', 'EQP_NAME', 'PRODG1', 'TOOL_NAME'.")

        try:
            merge_operno = list(parse_dict[0]['mergeOperno']) if parse_dict[0].get('mergeOperno') else None
        except KeyError:
            merge_operno = None

        try:
            merge_prodg1 = list(parse_dict[0]['mergeProdg1']) if parse_dict[0].get('mergeProdg1') else None
        except KeyError:
            merge_prodg1 = None

        try:
            merge_product = list(parse_dict[0]['mergeProductId']) if parse_dict[0].get('mergeProductId') else None
        except KeyError:
            merge_product = None

        try:
            merge_eqp = list(parse_dict[0]['mergeEqp']) if parse_dict[0].get('mergeEqp') else None
        except KeyError:
            merge_eqp = None

        try:
            merge_chamber = list(parse_dict[0]['mergeChamber']) if parse_dict[0].get('mergeChamber') else None
        except KeyError:
            merge_chamber = None
        return parse_dict, request_id, grpby_list, merge_operno, merge_prodg1, merge_product, merge_eqp, merge_chamber

    @staticmethod
    def run(df_info: pd.DataFrame, df1: pyspark.sql.dataframe):
        try:
            parse_dict, request_id, grpby_list, merge_operno, merge_prodg1, merge_product, merge_eqp, merge_chamber = ExertUvaAlgorithm.get_some_info(
                df_info)
            # todo1: 将parse_dict传给解析SQL的函数, 拼接SQL获取数据

            # 1. Station merge and data preprocessing
            df_integrate_columns = PreprocessForUvaData.integrate_columns(df=df1,
                                                                          merge_operno_list=merge_operno,
                                                                          merge_prodg1_list=merge_prodg1,
                                                                          merge_product_list=merge_product,
                                                                          merge_eqp_list=merge_eqp,
                                                                          merge_chamber_list=merge_chamber)
            m, n = df_integrate_columns.count(), len(df_integrate_columns.columns)
            print(f"Merged data: ({m}, {n})")
            if df_integrate_columns.count() == 0:
                msg = 'Station merge exception!！'
                return msg

            df_run = PreprocessForUvaData.pre_process(df_integrate_columns)
            m, n = df_run.count(), len(df_run.columns)
            print(f"Preprocessed data: ({m}, {n})")
            if df_run.count() == 0:
                msg = 'No data in the database under this condition！'
                return msg

            # 2. Commonality analysis
            common_res = PreprocessForUvaData.commonality_analysis(df_run=df_run, grpby_list=grpby_list)
            common_res.show()
            if common_res.count() == 0:
                msg = 'Commonality analysis result exception!'
                return msg

            data_dict_list_bs = PreprocessForUvaData.get_data_list(common_res=common_res, grpby_list=grpby_list,
                                                                   big_or_small='big')
            print("data_dict_list_bs:", data_dict_list_bs)
            print("len(data_dict_list_bs):", len(data_dict_list_bs))

            if len(data_dict_list_bs) != 0:
                print("****************Call Big Sample Algorithm****************")
                result = ExertUvaAlgorithm.fit_big_data_model(df_run=df_run, data_dict_list_bs=data_dict_list_bs,
                                                              grpby_list=grpby_list, request_id=request_id)
            else:
                print("****************Call Small Sample Algorithm****************")
                result = ExertUvaAlgorithm.fit_small_data_model(df_run=df_run, common_res=common_res,
                                                                grpby_list=grpby_list, request_id=request_id)

            if isinstance(result, str):
                return result
            else:
                # todo2: result 需要写入数据库的正确代码
                result.show()
                # engine = create_engine("mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(user="root", password="123456",
                #                                                                                          host="192.168.13.17",
                #                                                                                          port=9030,
                #                                                                                          db="rca"))
                # result.to_sql('uva_results', con=engine, index=False, if_exists='append')

        except Exception as e:
            result = str(e)
            return result

In [7]:
json_loads_dict = {
        "requestId": "uva",
        "requestParam": [
            {'dateRange': [{'start': "2023-12-01 00:00:00", 'end': "2024-01-15 00:00:00"}],
             'lot': [],
             'operNo': ["1G.EEG1R", "1G.PPB10"],
             'prodg1': [],
             'productId': [],
             'eqp': [],
             'tool': [],
             'recipeName': [],
             'waferId': {'good': ["NBX392-15", "NBX392-20", "NBX392-24", "NBX391-24", "NBX391-25", "NBX548-09",
                                  "NBX391-01", "NBX391-02", "NBX391-13", "NBX391-17"],
                         'bad': ["NBX500-10", "NBX500-01", "NBX500-09"]},
             'uploadId': '20240110170016023',
             'grpby_list': ["PRODG1", "OPER_NO", "EQP_NAME", "TOOL_NAME"],
             'mergeOperno': [],
             'mergeProdg1': [],
             'mergeProductId': [],
             'mergeEqp': [],
             'mergeChamber': [],
             }
        ]
    }

In [8]:
df_info_ = pd.DataFrame({"requestId": [json_loads_dict["requestId"]],
                         "requestParam": [json.dumps(json_loads_dict["requestParam"])]})

In [9]:
params_dict = {'df_info': df_info_, 'df1': df1_}

In [10]:
result_ = ExertUvaAlgorithm.run(**params_dict)
print(type(result_))
print("最后结果是：")
print(result_)

Merged data: (25276, 15)
Preprocessed data: (13780, 11)
+--------+--------+--------+---------------+-----------+--------+-------+
|  PRODG1| OPER_NO|EQP_NAME|      TOOL_NAME|wafer_count|good_num|bad_num|
+--------+--------+--------+---------------+-----------+--------+-------+
|L2800Z2N|1V.PPB10|   PBT01|PBT01_CGHA_4-14|          4|       1|      3|
|L2800Z2N|1V.PPB10|   PBT01|PBT01_CLHA_4-12|          4|       1|      3|
|L2800Z2N|1V.PPB10|   PBT01|PBT01_CLHA_4-21|          2|       0|      2|
|L2800Z2N|1V.PPB10|   PBT01|PBT01_CGHA_4-24|          2|       0|      2|
+--------+--------+--------+---------------+-----------+--------+-------+

data_dict_list_bs: []
len(data_dict_list_bs): 0
****************Call Small Sample Algorithm****************
<class 'str'>
最后结果是：
Temporary algorithm exception!


In [26]:
prodg1, oper, eqp, tool = 'L2800Z2N', '1V.PPB10', 'PBT01', 'PBT01_CGHA_4-14'

In [20]:
prodg1, oper, eqp, tool = 'L2800Z2N', '1V.PPB10', 'PBT01', 'PBT01_CLHA_4-12'

In [22]:
prodg1, oper, eqp, tool = 'L2800Z2N', '1V.PPB10', 'PBT01', 'PBT01_CLHA_4-21'

In [24]:
prodg1, oper, eqp, tool = 'L2800Z2N', '1V.PPB10', 'PBT01', 'PBT01_CGHA_4-24'

In [27]:
df_pandass = pandas_df.query(f"PRODG1 == '{prodg1}' & OPER_NO == '{oper}' & EQP_NAME == '{eqp}' & TOOL_NAME == '{tool}'")

df_pandass

,WAFER_ID,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,LOT_ID,RECIPE_NAME,OPER_NO,parametric_name,CASE_INFO,STATUS,STATISTIC_RESULT,label
488,NBX221-13,9287,329563,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,NBX221.100,WaferFlow/PRD/PROCESS/1NJ500N0,1V.PPB10,PLATE_TEMP#HDB205C60S_MEAN#MEAN,2023/10/13,NORMAL,250,1
7478,NBX220-06,9287,323876,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,NBX220.150,WaferFlow/PRD/PROCESS/1NJ500N0,1V.PPB10,PLATE_TEMP#HDB205C60S_MEAN#MEAN,2023/9/28,NORMAL,250,1
7481,NBX272-19,9287,331771,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,NBX272.000,WaferFlow/PRD/PROCESS/1NJ500N0,1V.PPB10,PLATE_TEMP#HDB205C60S_MEAN#MEAN,2023/10/18,NORMAL,250,1
24786,NBX293-06,9287,362271,PBT01,AFPNR901N.0B0L,L2800Z2N,PBT01_CGHA_4-14,NBX293.200,WaferFlow/PRD/PROCESS/1NJ500N0,1V.PPB10,PLATE_TEMP#HDB205C60S_MEAN#MEAN,2023/12/20,NORMAL,250,0
24787,NBX293-06,9287,362271,PBT01,AFPNR901N.0B0L,L2800Z2N,PBT01_CGHA_4-14,NBX293.200,WaferFlow/PRD/PROCESS/1NJ500N0,1V.PPB10,PLATE_TEMP#HDB205C60S_MEAN#MEAN,2023/12/20,NORMAL,250,0
